In [ ]:
#!/usr/bin/env python
# coding: utf-8
#######################################################################
# How to setup AndroSensor:
# Install AndroSensor
# Ensure when installing, allow app to run in background.
# Open settings and change recording interval to fast.
#
#
#
#######################################################################
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
import numpy as np
import scipy as sp
import scipy.signal as sg

px.set_mapbox_access_token(open(".mapbox_token").read())

In [ ]:
# IF useing ; sep, then delete the first line.
path = r'phone_data'  # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = [pd.read_csv(f, sep=";") for f in all_files]
concatenated_df = pd.concat(df_from_each_file, ignore_index=True, sort=True)

# Convert datatime to actual datetime
concatenated_df["timestamp"] = pd.to_datetime(concatenated_df["YYYY-MO-DD HH-MI-SS_SSS"],format='%Y-%m-%d %H:%M:%S:%f')
concatenated_df["Speed(km/h)"] = concatenated_df["LOCATION Speed ( Kmh)"]

In [ ]:
# calculate abs accleration
concatenated_df["abs_acceleration"] = np.linalg.norm(
    concatenated_df[["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"]].values, axis=1)

# Calculate abs location
concatenated_df["abs_loc"] = np.linalg.norm(concatenated_df[['LOCATION Latitude : ', 'LOCATION Longitude : ']].values,
                                            axis=1)

# Parse GPS sats properly
concatenated_df['Satellites in range'] = concatenated_df['Satellites in range'].str.extract(r'(\d*) \/ \d*')
concatenated_df['Satellites in range'] = pd.to_numeric(concatenated_df['Satellites in range'] )

# Display 3-axis acceleration vs time

In [ ]:
# Display accelration
fig = px.line(concatenated_df,
              x="timestamp",
              y=["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"],
              title='Acceleration'
             )
fig.write_html("plot_acc.html")

fig.show()

In [ ]:
cols = ["ACCELEROMETER X (m/s²)",
        "ACCELEROMETER Y (m/s²)",
        "ACCELEROMETER Z (m/s²)",
        "LOCATION Accuracy ( m)",
        "LOCATION Altitude ( m)",
        "LOCATION Altitude-google ( m)",
        "LOCATION Latitude : ",
        "LOCATION Longitude : ",
        "LOCATION ORIENTATION (°)",
        "LOCATION Speed ( Kmh)",
        "PROXIMITY (i)",
        "SOUND LEVEL (dB)",
        "Satellites in range",
        "timestamp"]

concatenated_df[cols]

# Upload all data to mongodb

In [ ]:
from pymongo import MongoClient
import urllib.parse

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)

collection = client["ebike_data"]["data"]

data = concatenated_df[cols].to_dict(orient='records')  # Here's our added param..

#collection.insert_many(data)

# Now process data from display GPS positions

In [ ]:
# remove all duplicated long lat positions
unique_coords_df = concatenated_df.groupby('abs_loc').last().reset_index()
unique_coords_df = unique_coords_df.sort_values(by='timestamp')

In [ ]:
unique_coords_df["gps_acceleration"] = unique_coords_df["LOCATION Speed ( Kmh)"].diff()
unique_coords_df["power"] = unique_coords_df["gps_acceleration"] * unique_coords_df["LOCATION Speed ( Kmh)"] * 80

#unique_coords_df.to_csv(
#    r"D:\OneDrive - Imperial College London\University Storage\Masters project\Raw data\combined.csv")

In [ ]:
unique_coords_df["slope"] = unique_coords_df["LOCATION Altitude ( m)"].diff()
#unique_coords_df["slope"].describe()

In [ ]:
#unique_coords_df["slope"].mask(unique_coords_df["slope"].between(-200, -10), inplace=True)
#unique_coords_df["slope"].mask(unique_coords_df["slope"].between(10, 200), inplace=True)

In [ ]:
unique_coords_df.dropna(subset = ['Speed(km/h)'], inplace = True) 

In [ ]:
# Display GPS positions
fig2 = px.scatter_mapbox(unique_coords_df,
                        lat="LOCATION Latitude : ",
                        lon="LOCATION Longitude : ",
                        color="Speed(km/h)",#"slope",#"LOCATION Altitude ( m)",,#"Speed(km/h)", # "abs_acceleration" or "gps_acceleration" or "power"
                        zoom=14,
                        hover_data=["YYYY-MO-DD HH-MI-SS_SSS", "LOCATION Altitude ( m)"],
                        #size="LOCATION Accuracy ( m)"
                       )


fig = px.line_mapbox(unique_coords_df,
                      lat="LOCATION Latitude : ",
                      lon="LOCATION Longitude : ")
fig.add_trace(fig2.data[0]) # adds the line trace to the first figure



fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("plot.html")
fig.show()

# Filter slope values

In [ ]:
# We get a triangular window with 60 samples.
h = sg.get_window('triang', 60)
# We convolve the signal with this window.
unique_coords_df["fil"] = sg.convolve(unique_coords_df["slope"], h / h.sum())

# Display slope vs time

In [ ]:
# Display accelration
fig = px.line(unique_coords_df,
              x="YYYY-MO-DD HH-MI-SS_SSS",
              y=["LOCATION Altitude ( m)"],
              #y=["slope"],

              title='altitude'
             )
fig.write_html("plot_slope.html")
fig.show()